In [493]:
import torch
from torchvision import datasets, transforms
from torch import nn
from torch.utils.data import Dataset

import numpy as np
from matplotlib import pyplot as plt

(i)knowledge-oblivious–the attacker shall have no knowledge of the target
model’s parameters/structures, nor the original training datasets, 

(ii) cleanlabel–the attacker shall not be able to control the labeling process, and

(iii)clean-test–test-time instances shall not be required to be modified using added
adversarial perturbations for attacking effectiveness

https://www.ecva.net/papers/eccv_2020/papers_ECCV/papers/123720137.pdf

Make sure that you do not reuse the data for training, testing and the attack/defence. - prof

In [497]:
class FashionDataset(Dataset):
    """User defined class to build a datset using Pytorch class Dataset."""
    
    def __init__(self,X,Y, transform = None):
        """Method to initilaize variables.""" 
        self.images = X
        self.labels = Y
        self.transform = transform
        

    def __getitem__(self, index):
        label = self.labels[index]
        image = self.images[index]
        
        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def __len__(self):
        return len(self.images)

In [498]:
transform = transforms.Compose([
                                # transforms.ToPILImage(),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,)),
                           
                               ])
trainset = datasets.FashionMNIST('MNIST_data/', download = True, train = True,transform=transform)
testset = datasets.FashionMNIST('MNIST_data/', download = True, train = False,transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 12, shuffle = True)
testloader = torch.utils.data.DataLoader(testset, batch_size = 12, shuffle = True)

<h3>Infecting the data</h3>

In [499]:
x_train, y_train = trainset.data, trainset.targets

Seperating out the indices for the normal and the infected

In [500]:
size_of_infected=30000

In [501]:
#generate the infected random indices from the training sample
generatePoisonIdx = torch.from_numpy(np.random.choice(len(x_train), size=(size_of_infected,), replace=False)) #poison index

In [502]:
#generate normal indices excluding from the training sample
trainIdx = torch.tensor([i for i in range(len(x_train)) if i not in generatePoisonIdx]) #true index

In [503]:
#indexing all the infected
x_pois_train, y_pois_train = x_train[generatePoisonIdx],y_train[generatePoisonIdx]

In [504]:
#indexing all the noromal
x_train_new, y_train_new = x_train[trainIdx],y_train[trainIdx]

In [505]:
#mix up the labels(infecting the infected) 
for index,value in enumerate(y_pois_train):
    randomIdx = np.random.choice(10)
    if randomIdx!=value.item():
        y_pois_train[index]=randomIdx
    else:
        randomIdx = np.random.choice(10)
        y_pois_train[index]=randomIdx

Merging the normal and infected

In [506]:
x_poison_train = torch.cat([x_train_new,x_pois_train], dim=0)
y_poison_train = torch.cat([y_train_new,y_pois_train],dim=0)

In [ ]:
#final data

In [507]:
print("Training samples after infection : ",x_poison_train.size())
print("Labels samples after infection : ",y_poison_train.size())

Training samples after infection :  torch.Size([60000, 28, 28])
Labels samples after infection :  torch.Size([60000])


In [508]:
transform_new = transforms.Compose([
                                transforms.ToPILImage(),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,)),
                           
                               ])

In [509]:
dataset = FashionDataset(x_poison_train,y_poison_train,transform_new)

In [510]:
trainloader = torch.utils.data.DataLoader(dataset, batch_size = 12, shuffle = True)
testloader = torch.utils.data.DataLoader(testset, batch_size = 12, shuffle = True)

In [472]:
def plot(x):
    # print(x.numpy().shape)
    plt.imshow(x.numpy().reshape(28,28),cmap="gray")

In [489]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [511]:
model = nn.Sequential(nn.Linear(784, 256),
                      nn.ReLU(),
                      nn.Dropout(0.2),
                      nn.Linear(256, 128),
                      nn.ReLU(),
                      nn.Dropout(0.2),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Dropout(0.2),
                      nn.Linear(64, 10),
                      nn.LogSoftmax(dim = 1)
      
                ).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.002)
criterion = nn.NLLLoss()

In [512]:
train_losses, test_losses = [], []
epochs = 10


for e in range(epochs):
  running_loss = 0
  for images, labels in trainloader:
    # Flatten Fashion-MNIST images into a 784 long vector
    images = images.view(images.shape[0], -1).to(device)
    images.requires_grad = True
    labels = labels.to(device)
    # Training pass
    optimizer.zero_grad()
    
    output = model.forward(images)
    loss = criterion(output, labels)
    loss.backward()
    optimizer.step()
    
    running_loss += loss.item()
  else:
    test_loss = 0
    accuracy = 0
    
    # Turn off gradients for validation, saves memory and computation
    with torch.no_grad():
      # Set the model to evaluation mode
      model.eval()
      
      # Validation pass
      for images, labels in testloader:
        images = images.view(images.shape[0], -1).to(device)
        labels = labels.to(device)
        log_ps = model(images)
        test_loss += criterion(log_ps, labels)
        
        ps = torch.exp(log_ps)
        top_p, top_class = ps.topk(1, dim = 1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor))
    
    model.train()
    train_losses.append(running_loss/len(trainloader))
    test_losses.append(test_loss/len(testloader))
    
    print("Epoch: {}/{}..".format(e+1, epochs),
          "Training loss: {:.3f}..".format(running_loss/len(trainloader)),
          "Test loss: {:.3f}..".format(test_loss/len(testloader)),
          "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))

Epoch: 1/10.. Training loss: 2.069.. Test loss: 1.324.. Test Accuracy: 0.765
Epoch: 2/10.. Training loss: 2.018.. Test loss: 1.366.. Test Accuracy: 0.777
Epoch: 3/10.. Training loss: 2.010.. Test loss: 1.308.. Test Accuracy: 0.778
Epoch: 4/10.. Training loss: 2.005.. Test loss: 1.246.. Test Accuracy: 0.777
Epoch: 5/10.. Training loss: 2.007.. Test loss: 1.249.. Test Accuracy: 0.781
Epoch: 6/10.. Training loss: 2.005.. Test loss: 1.264.. Test Accuracy: 0.791
Epoch: 7/10.. Training loss: 2.005.. Test loss: 1.284.. Test Accuracy: 0.781
Epoch: 8/10.. Training loss: 2.004.. Test loss: 1.317.. Test Accuracy: 0.800
Epoch: 9/10.. Training loss: 2.025.. Test loss: 1.194.. Test Accuracy: 0.792
Epoch: 10/10.. Training loss: 2.006.. Test loss: 1.304.. Test Accuracy: 0.785
